In [1]:
from scipy.io import wavfile
from scipy.fft import rfft, rfftfreq, irfft
import numpy as np
from tqdm import tqdm

## Read data

In [2]:
samplerate, data = wavfile.read("enbateau.wav")
N = data.shape[0]
length = N / samplerate

In [3]:
left = data[:, 0]
right = data[:, 1]

## Real FFT

In [4]:
y_left = rfft(left)
y_right = rfft(right)

In [5]:
f = rfftfreq(N, 1. / samplerate)

## Filter and Inverse RFFT

In [6]:
def truncate(min, max):
    y_left_t = y_left.copy()
    y_left_t[abs(f) < min] = 0
    y_left_t[abs(f) > max] = 0

    y_right_t = y_right.copy()
    y_right_t[abs(f) < min] = 0
    y_right_t[abs(f) > max] = 0

    return y_left_t, y_right_t

In [7]:
def process(min, max, filename = None):
    y_left_t, y_right_t = truncate(min, max)
    left_t = irfft(y_left_t)
    right_t = irfft(y_right_t)
    data_t = np.stack((left_t, right_t), axis = -1)
    if filename is None:
        filename = "{}-{}.wav".format(min, max)
    wavfile.write(filename, samplerate, data_t.astype(np.int16))

## Save
[Frequency range chart](https://upload.wikimedia.org/wikipedia/commons/5/5d/Animal_hearing_frequency_range.svg)

In [8]:
animals = (("tuna",50,1100),("chicken",125,2000),("goldfish",20,3000),("bullfrog",100,3000),("catfish",50,4000),("tree frog",50,4000),("canary",250,8000),("cockatiel",250,8000),("parakeet",200,8500),("elephant",17,10500),("owl",200,12000),("human",31,19000),("chinchilla",52,33000),("horse",55,33500),("cow",23,35000),("raccoon",100,40000),("sheep",125,42500),("dog",64,44000),("ferret",16,44000),("hedgehog",250,45000),("guinea pig",47,49000),("rabbit",96,49000),("sea lion",200,50000),("gerbil",56,60000),("opossum",500,64000),("albino rat",390,72000),("hooded rat",530,75000),("cat",55,77000),("mouse",900,79000),("little brown bat",10300,115000),("beluga whale",1000,123000),("bottlenose dolphin",150,150000),("porpoise",75,150000))

In [9]:
for animal in tqdm(animals):
    process(animal[1], animal[2], filename = animal[0] + ".wav")

100%|██████████| 33/33 [02:01<00:00,  3.69s/it]
